# 3.8.1.4. daum_news_list_final

In [1]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os
import time

In [2]:
def db_save(NEWS_LIST):
    with sqlite3.connect(os.path.join('.', 'sqliteDB')) as con:
        try:
            NEWS_LIST.to_sql(name = 'NEWS_LIST', con = con, index = False, if_exists = 'append')
            # if_exists : {'fail', 'replce', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(NEWS_LIST), '건 저장완료..')

In [3]:
def db_delete():
    with sqlite3.connect(os.path.join('.', 'sqliteDB')) as con:
        try:
            cur = con.cursor()
            sql = 'DELETE FROM NEWS_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e))

In [4]:
def db_select():
    with sqlite3.connect(os.path.join('.', 'sqliteDB')) as con:
        try:
            query = 'SELECT * FROM NEWS_LIST'
            NEWS_LIST = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e))
        return NEWS_LIST

In [5]:
import re
import string

def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
        if p.text: # 체크
            body.append(re.sub(punc, '', p.text)) # 특수문자 제거
    full_body = ' '.join(body)
    
    return full_body

In [19]:
page = 58
max_page = 0
REG_DATE = '20220719'

In [20]:
while(True):
    df_list = []
    response = requests.get('http://news.daum.net/breakingnews/digital?page={}&regDate={}'\
                           .format(page, REG_DATE))
    root = lxml.html.fromstring(response.content)
    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a = li.xpath('div/strong/a')[0]
        url = a.get('href')
        article = get_detail(url)
        df = pd.DataFrame({'URL' : [url], 'TITLE':[a.text], 'ARTICLE' : [article]})
        df_list.append(df)
    
    if df_list:
        df_10 = pd.concat(df_list)
        db_save(df_10)
        
    # 페이지 번호 중에서 max 페이지 가져오기
    for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'):
        try:
            num = int(a.text)
            if max_page < num:
                max_page = num
        except:
            pass
        
    
    # 마지막 페이지 여부 확인        
    span = root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[@clss="btn_page btn_next"]')
    
    if(len(span) <= 0) & (page > max_page):
        break
    else:
        page = page + 1
    
    time.sleep(1)

In [21]:
# db_delete()

In [22]:
print(db_select())

                                      URL  \
0  https://v.daum.net/v/20220718030251882   
1  https://v.daum.net/v/20220718030110781   
2  https://v.daum.net/v/20220718030031766   
3  https://v.daum.net/v/20220718000343281   
4  https://v.daum.net/v/20220718000148237   
5  https://v.daum.net/v/20220718000019204   

                                           TITLE  \
0         "우리 지역서 원격근무하면 1만2000달러 그냥 드릴게" [핫 코너]   
1  반도체 패키지 기판 수요 폭발.. 26만㎡ 공장에 3000억 투입해 라인 증설 중   
2               삼성·배민·토스.. 독서광 CEO들 "이 책은 꼭 읽어라"   
3                             [사진] 로봇이 알아서 운송 척척   
4     숏폼이 대세..유튜브 "쇼츠 조회수 하루 300억회, 수익화 방안 곧 마련"   
5                  보건복지부, 백신‧바이오의약품 생산공정 기본교육 진행   

                                             ARTICLE  
0  한때 미 석유산업의 중심지로 ‘오일 도시’라 불렸던 중서부 오클라호마주 털사는 지난...  
1  지난 14일 오후 부산 강서구 녹산산단에 있는 삼성전기 부산사업장 4500명이 근무...  
2  대기업스타트업 CEO최고경영자가 ‘올여름 휴가철 추천하고픈 책’은 무엇일까 본지의 ...  
3   지난 15일 경기도 군포시 CJ대한통운 스마트 풀필먼트 센터에서 AGV바닥에 부착...  
4  세계 최대 동영상 플랫폼 유튜브가 숏폼15초 내외의 짧은 영상 콘텐트인 ‘